In [ ]:
!pip install python-docx

In [ ]:
!pip install docxtpl

In [ ]:
!pip install dataframe_image

In [1]:
import pandas as pd
import numpy as np
from plotnine import *
from statsmodels.stats.proportion import proportions_ztest
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Mm
import jinja2
from scipy.stats import fisher_exact
from mizani.formatters import percent_format
import docx
import os
import dataframe_image as dfi
import warnings
warnings.filterwarnings('ignore')

In [2]:
#bli = pd.read_excel('Kantar Norms_Amazon 2022.01.26.xlsx', sheet_name='BLI Campaigns')
bli = pd.read_excel('KANTAR_FIVERR_Kantar Norms 2022.02.25.xlsx', sheet_name='BLI Campaigns')

In [3]:
# 80% threshold for sig level, should interpret this as power of the test: 80% of the change that
# the test will correctly reject the null hypothesis. 80% is a typical threshold used in the brand lift study 
# including Kantar BLI
# We only keep records 
bli = bli[bli['Sig level'] >= 0.8]
bli.shape

(1298, 29)

In [4]:
bli['Question type'] = bli['Normalized Norms']
bli['Upload Quarter'] = bli['Upload Quarter'].str.replace(r"['\s]", '')
bli['Campaign'] = bli['Campaign'].str.replace("'| ","")


In [5]:
# BLI Campaign: 2021
qtr_flag_bli = bli['Upload Quarter'].isin(["Q121","Q221","Q321","Q421"])
vertical_flag_bli = bli['Test Vertical'].isin(['Financial Services'])
bli = bli[(qtr_flag_bli) & (vertical_flag_bli)][['Upload Quarter','Campaign','Country','Question type',\
                                                 'Test Vertical','Control n','Control','Exposed n','Exposed']]

In [6]:
bli

,Upload Quarter,Campaign,Country,Question type,Test Vertical,Control n,Control,Exposed n,Exposed
2116,Q121,American Express UK Q'21,UK,Unaided Awareness,Financial Services,100,0.394,99,0.421
2219,Q221,Autotrader OTT Q1'21,US,Unaided Awareness,Financial Services,250,0.061,250,0.109
2222,Q221,Autotrader OTT Q1'21,US,Intent (NET),Financial Services,250,0.378,250,0.473
2284,Q221,GEICO Quirky Family Sponsorship 2021,US,Ad Awareness,Financial Services,251,0.733,250,0.788
2285,Q221,GEICO Quirky Family Sponsorship 2021,US,Message Association,Financial Services,251,0.065,250,0.126
2420,Q221,SumUp FireTV UK Q2'21,UK,Aided Awareness,Financial Services,231,0.130,235,0.188
2421,Q221,SumUp FireTV UK Q2'21,UK,Ad Awareness,Financial Services,231,0.043,235,0.094
2422,Q221,SumUp FireTV UK Q2'21,UK,Brand Favorability,Financial Services,231,0.075,235,0.121
2459,Q221,Progressive IMDb TV Sponsorship April Q2'21 BLI,US,Consideration Intent,Financial Services,250,0.527,235,0.617
2530,Q221,Mastercard CA Q2'21,CA,Unaided Awareness,Financial Services,250,0.576,249,0.229


In [7]:
def brand_lift_test(df, alternative='two-sided'):
    # In two samples case, count and obs should be numpy array
    
    stat_list = []
    p_value_list = []
    
    for _, row in df.iterrows():
       
        df_temp = pd.DataFrame({'control':[row['Control n'] * row['Control'],\
                                           row['Control n'] * (1 - row['Control'])],\
                                'test':[row['Exposed n'] * row['Exposed'], row['Exposed n'] * (1-row['Exposed'])]})
    
        stat, p_value = fisher_exact(df_temp.to_numpy(),  alternative = alternative)
        stat_list.append(stat)
        p_value_list.append(p_value)
    
    df['stat'] = stat_list
    df['p_value'] = p_value_list
    
    return df

def weighted_average(df,group_cols):
    return np.average(df[group_cols[0]], weights=df[group_cols[1]])

In [8]:

survey_num_agg = bli.groupby(['Test Vertical','Question type']).agg({'Control n':sum,'Exposed n':sum})

control_agg= \
bli.groupby(['Test Vertical','Question type']).apply(weighted_average,group_cols=['Control','Control n'])
exposed_agg= \
bli.groupby(['Test Vertical','Question type']).apply(weighted_average,group_cols=['Exposed','Exposed n'])

bli_agg = pd.concat([survey_num_agg,control_agg,exposed_agg],axis=1)
bli_agg.rename(columns={0:'Control',1:'Exposed'}, inplace=True)

In [9]:
bli_agg

Control n  Exposed n   Control  \
Test Vertical      Question type                                          
Financial Services Ad Awareness               2275       2341  0.267791   
                   Aided Awareness            1280       1281  0.331685   
                   Brand Favorability         1081       1021  0.338043   
                   Consideration Intent       1396       1323  0.394953   
                   Intent (NET)                749        750  0.409220   
                   Message Association         801        794  0.087722   
                   Purchase Intent             250        253  0.399000   
                   Unaided Awareness          2438       2510  0.363927   

                                          Exposed  
Test Vertical      Question type                   
Financial Services Ad Awareness          0.339059  
                   Aided Awareness       0.442178  
                   Brand Favorability    0.375742  
                   Consideration Intent  0.384134  
                   Intent (NET)          0.360667  
                   Message Association   0.120329  
                   Purchase Intent       0.330000  
                   Unaided Awareness     0.367480

In [10]:
#bli_result = brand_lift_test(bli)
bli_result = brand_lift_test(bli_agg)
print(bli_result)

                                         Control n  Exposed n   Control  \
Test Vertical      Question type                                          
Financial Services Ad Awareness               2275       2341  0.267791   
                   Aided Awareness            1280       1281  0.331685   
                   Brand Favorability         1081       1021  0.338043   
                   Consideration Intent       1396       1323  0.394953   
                   Intent (NET)                749        750  0.409220   
                   Message Association         801        794  0.087722   
                   Purchase Intent             250        253  0.399000   
                   Unaided Awareness          2438       2510  0.363927   

                                          Exposed      stat       p_value  
Test Vertical      Question type                                           
Financial Services Ad Awareness          0.339059  0.713543  1.766647e-07  
                   Ai

In [11]:
# replace white space in the column name because accessing column name with white space might
# not be possible using docxtpl
bli_result['Lift'] = round((bli_result['Exposed'] - bli_result['Control'])*100,2).astype(str) + '%'
bli_result['Sig'] = bli_result['p_value'].apply(lambda x: 'Significant' if x < 0.05 else 'Not Significant')
bli_result.reset_index(inplace=True)

In [12]:
bli_result

,Test Vertical,Question type,Control n,Exposed n,Control,Exposed,stat,p_value,Lift,Sig
0,Financial Services,Ad Awareness,2275,2341,0.267791,0.339059,0.713543,1.766647e-07,7.13%,Significant
1,Financial Services,Aided Awareness,1280,1281,0.331685,0.442178,0.625578,1.006952e-08,11.05%,Significant
2,Financial Services,Brand Favorability,1081,1021,0.338043,0.375742,0.849039,7.550280e-02,3.77%,Not Significant
3,Financial Services,Consideration Intent,1396,1323,0.394953,0.384134,1.046092,5.817589e-01,-1.08%,Not Significant
4,Financial Services,Intent (NET),749,750,0.409220,0.360667,1.228205,5.591509e-02,-4.86%,Not Significant
5,Financial Services,Message Association,801,794,0.087722,0.120329,0.704542,3.960822e-02,3.26%,Significant
6,Financial Services,Purchase Intent,250,253,0.399000,0.330000,1.343855,1.154603e-01,-6.9%,Not Significant
7,Financial Services,Unaided Awareness,2438,2510,0.363927,0.367480,0.985004,8.132876e-01,0.36%,Not Significant


In [13]:
# Bli
bli_id_columns = bli_agg.columns[~bli_agg.columns.isin(['Control','Exposed'])]
bli_long = bli_result.melt(id_vars=bli_id_columns, value_vars=['Control','Exposed'],\
                           var_name='Group', value_name='Score')
bli_long['Score_Text'] = round(bli_long['Score'] * 100,2).astype(str) + "%"

In [14]:
bli_long

,Test Vertical,Question type,Control n,Exposed n,stat,p_value,Lift,Sig,Group,Score,Score_Text
0,Financial Services,Ad Awareness,2275,2341,0.713543,1.766647e-07,7.13%,Significant,Control,0.267791,26.78%
1,Financial Services,Aided Awareness,1280,1281,0.625578,1.006952e-08,11.05%,Significant,Control,0.331685,33.17%
2,Financial Services,Brand Favorability,1081,1021,0.849039,7.550280e-02,3.77%,Not Significant,Control,0.338043,33.8%
3,Financial Services,Consideration Intent,1396,1323,1.046092,5.817589e-01,-1.08%,Not Significant,Control,0.394953,39.5%
4,Financial Services,Intent (NET),749,750,1.228205,5.591509e-02,-4.86%,Not Significant,Control,0.409220,40.92%
5,Financial Services,Message Association,801,794,0.704542,3.960822e-02,3.26%,Significant,Control,0.087722,8.77%
6,Financial Services,Purchase Intent,250,253,1.343855,1.154603e-01,-6.9%,Not Significant,Control,0.399000,39.9%
7,Financial Services,Unaided Awareness,2438,2510,0.985004,8.132876e-01,0.36%,Not Significant,Control,0.363927,36.39%
8,Financial Services,Ad Awareness,2275,2341,0.713543,1.766647e-07,7.13%,Significant,Exposed,0.339059,33.91%
9,Financial Services,Aided Awareness,1280,1281,0.625578,1.006952e-08,11.05%,Significant,Exposed,0.442178,44.22%


In [15]:
bli_list = []

for bli_name, bli_df in bli_long.groupby(['Test Vertical','Question type']):
    bli_plot = ggplot(bli_df, aes(x="Group", y="Score", fill="Group", label="Score_Text"))\
    + geom_bar(stat = "identity", position = "dodge") + ggtitle(bli_name[0] + "-" + bli_name[1])\
    + xlab('Audience Group') + ylab('Score Percentage') + theme(legend_title=element_blank()) \
    + geom_text(position=position_stack(vjust=1)) \
    + scale_y_continuous(labels=percent_format()) 
    bli_list.append({'Test Vertical':bli_name[0],
                     'Question type':bli_name[1],
                     'bar_plot':bli_plot})

bli_group = pd.DataFrame(bli_list)
bli_result = bli_result.merge(bli_group, how='inner', on=['Test Vertical', 'Question type'])
bli_result.columns = [c.replace(' ','_') for c in bli_result.columns]

In [16]:
bli_result['Question_type'].unique()

array(['Ad Awareness', 'Aided Awareness', 'Brand Favorability',
       'Consideration Intent', 'Intent (NET)', 'Message Association',
       'Purchase Intent', 'Unaided Awareness'], dtype=object)

## Reporting

In [17]:
if not os.path.exists('img/'):
    os.mkdir('img/')
    
for idx, row in bli_result.iterrows():
    img_path =  'img/bli_plot' + str(idx) +'.png'
    row['bar_plot'].save(img_path)


In [18]:
# test image
doc = DocxTemplate("Brand Lift Test Template.docx")
bli_experiments = []

for idx in bli_result.index:
    bli_experiments.append({'Test_Vertical': bli_result.iloc[idx]['Test_Vertical'],\
                        'Question_type': bli_result.iloc[idx]['Question_type'],\
                        'Lift': bli_result.iloc[idx]['Lift'],\
                        'p_value': bli_result.iloc[idx]['p_value'],\
                        'Sig':bli_result.iloc[idx]['Sig'],\
                        'Control_n': bli_result.iloc[idx]['Control_n'],\
                        'Exposed_n': bli_result.iloc[idx]['Exposed_n'],\
                        'image':InlineImage(doc,'img/bli_plot' + str(idx) +'.png', width=Mm(120), height=Mm(80))})
    
# sort by p-value in ascending order    
bli_experiments = sorted(bli_experiments, key = lambda i: (i['Test_Vertical'],i['p_value']))

In [19]:
#
if sum(bli_result.columns.isin(['bar_plot'])) > 0:
    bli_result.drop(columns='bar_plot', inplace=True)

bli_agg = bli_agg.sort_values('p_value', ascending=True)
dfi.export(bli_agg,"img/df_agg.png")
q_num = sum(bli_agg['Sig'] == 'Significant')
q_denom = bli_agg['Question type'].nunique()

sig_lift = round(bli_agg.groupby('Sig').apply(lambda x: np.mean(x['Exposed'] - x['Control']))['Significant'] * 100,2)
sig_control_n = round(bli_agg.groupby('Sig').apply(lambda x: np.mean(x['Control n']))['Significant'])
sig_exposed_n = round(bli_agg.groupby('Sig').apply(lambda x: np.mean(x['Exposed n']))['Significant'])
sig_sample = round(bli_agg.groupby('Sig').apply(lambda x: np.mean(x['Control n'] + x['Exposed n']))['Significant'])

In [20]:
bli_agg

,Test Vertical,Question type,Control n,Exposed n,Control,Exposed,stat,p_value,Lift,Sig
1,Financial Services,Aided Awareness,1280,1281,0.331685,0.442178,0.625578,1.006952e-08,11.05%,Significant
0,Financial Services,Ad Awareness,2275,2341,0.267791,0.339059,0.713543,1.766647e-07,7.13%,Significant
5,Financial Services,Message Association,801,794,0.087722,0.120329,0.704542,3.960822e-02,3.26%,Significant
4,Financial Services,Intent (NET),749,750,0.409220,0.360667,1.228205,5.591509e-02,-4.86%,Not Significant
2,Financial Services,Brand Favorability,1081,1021,0.338043,0.375742,0.849039,7.550280e-02,3.77%,Not Significant
6,Financial Services,Purchase Intent,250,253,0.399000,0.330000,1.343855,1.154603e-01,-6.9%,Not Significant
3,Financial Services,Consideration Intent,1396,1323,0.394953,0.384134,1.046092,5.817589e-01,-1.08%,Not Significant
7,Financial Services,Unaided Awareness,2438,2510,0.363927,0.367480,0.985004,8.132876e-01,0.36%,Not Significant


In [21]:
bli_agg_long = bli_agg.melt(id_vars=['Test Vertical','Question type'],\
                value_vars=['Control n','Exposed n','Control','Exposed'],\
                           var_name='Var', value_name='Value')

sample_plot = ggplot(bli_agg_long[bli_agg_long['Var'].isin(['Control n','Exposed n'])],\
       aes(x='Var', y='Value', fill='Question type')) + \
geom_bar(stat = "identity", position = "dodge") + \
labs(x='Audience Group', y='Number of Respondents',\
     fill='Question Type', title='Total Survey Respondents by Group and Question Type') + \
scale_x_discrete(labels=['Control','Exposed'])

sample_pct_plot = ggplot(bli_agg_long[bli_agg_long['Var'].isin(['Control','Exposed'])],\
       aes(x='Var', y='Value', fill='Question type')) + \
geom_bar(stat = "identity", position = "dodge") + \
labs(x='Audience Group', y='Average Response Score', \
     fill='Question Type',title='Average Survey Score by Group and Question Type') + \
scale_x_discrete(labels=['Control','Exposed']) + scale_y_continuous(labels=percent_format())

sample_plot.save('img/sample_plot.png')
sample_pct_plot.save('img/sample_pct_plot.png')

In [22]:
context = {}
context['bli_experiments'] = bli_experiments
context['img_df_agg'] = InlineImage(doc,'img/df_agg.png',width=Mm(180), height=Mm(40))
context['sample_plot'] = InlineImage(doc,'img/sample_plot.png',width=Mm(140), height=Mm(80))
context['sample_pct_plot'] = InlineImage(doc,'img/sample_pct_plot.png',width=Mm(140), height=Mm(80))
context['q_num'] = q_num
context['q_denom'] = q_denom
context['sig_sample'] = sig_sample
context['sig_control_n'] = sig_control_n
context['sig_exposed_n'] = sig_exposed_n
context['sig_lift'] = sig_lift
context['vertical'] = 'Financial Services'
jinja_env = jinja2.Environment(autoescape=True)
doc.render(context, jinja_env)
doc.save("BLI Campaign Brand Lift Study.docx")